In [26]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

In [71]:
df = pd.read_csv("/kaggle/input/cartoon-genre-classifer/final_preprocessed_data.csv")
df.head()

,Title,Description,Genre
0,Attack on Titan The Final Season: The Final Chapters,Final part of Attack on Titan The Final Season.,"['Action', 'Drama', 'Fantasy', 'Shounen', 'Dark Fantasy', 'Military', 'War', 'Based on a Manga']"
1,Fruits Basket the Final Season,The final arc of Fruits Basket.,"['Drama', 'Fantasy', 'Romance', 'Shoujo', 'Animal Transformation', 'Contemporary Fantasy', 'Curse', 'Dysfunctional Families', 'Mental Illness', 'Orphans', 'Zodiac', 'Based on a Manga']"
2,Fullmetal Alchemist: Brotherhood,"The foundation of alchemy is based on the law of equivalent exchange; you cannot produce something from nothing. As such, alchemy is bound by one taboo - human transmutation. Four years ago two young brothers, Edward and Alphonse Elric, broke this taboo when they tried to resurrect their dead mother. During the process Al's body disintegrated and Ed lost his leg. In a desperate attempt to prevent his brother from disappearing completely, Ed sacrificed one of his arms so he could affix Al's soul to a suit of armor. When his missing limbs are replaced by auto mail parts, Ed bears the name of...","['Action', 'Adventure', 'Drama', 'Fantasy', 'Mystery', 'Shounen', 'Conspiracy', 'Death of a Loved One', 'Military', 'Siblings', 'Based on a Manga']"
3,Demon Slayer: Kimetsu no Yaiba - Entertainment District Arc,"Tanjiro and his friends accompany the Hashira Tengen Uzui to an entertainment district where Tengen’s female ninja agents were gathering information on a demon before they suddenly disappeared. In order to investigate, Tanjiro and the others disguise themselves as women to sneak in!","['Action', 'Adventure', 'Shounen', 'Demons', 'Historical', 'Martial Arts', 'Orphans', 'Siblings', 'Supernatural', 'Swordplay', 'Based on a Manga']"
4,Bleach: Thousand-Year Blood War,"The peace is suddenly broken when warning sirens blare through the Soul Society. Residents there are disappearing without a trace and nobody knows who's behind it. Meanwhile, a dark shadow is also extending itself toward Ichigo and his friends in Karakura Town…","['Action', 'Comedy', 'Drama', 'Shounen', 'Afterlife', 'Ghosts', 'Shinigami', 'Supernatural', 'War', 'Based on a Manga']"


In [72]:
df = df.dropna().reset_index(drop=True)
df.shape

(24337, 3)

In [75]:
genres_list = df.Genre.to_list()
genre_count = {}
for genres in genres_list:
  genre_list = eval(genres)
  for genre in genre_list:
    if genre in genre_count.keys():
      genre_count[genre] += 1
    else:
      genre_count[genre] = 1
print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

Number of Genres: 655
{'Action': 7885, 'Drama': 4540, 'Fantasy': 6089, 'Shounen': 4957, 'Dark Fantasy': 83, 'Military': 1018, 'War': 184, 'Based on a Manga': 2431, 'Romance': 4322, 'Shoujo': 1166, 'Animal Transformation': 41, 'Contemporary Fantasy': 235, 'Curse': 42, 'Dysfunctional Families': 7, 'Mental Illness': 22, 'Orphans': 108, 'Zodiac': 8, 'Adventure': 4613, 'Mystery': 1325, 'Conspiracy': 41, 'Death of a Loved One': 43, 'Siblings': 123, 'Demons': 604, 'Historical': 1288, 'Martial Arts': 582, 'Supernatural': 3068, 'Swordplay': 182, 'Comedy': 9207, 'Afterlife': 21, 'Ghosts': 45, 'Shinigami': 17, 'Ancient China': 82, 'Chinese Animation': 547, 'Cultivation': 68, 'Xianxia': 36, 'Based on a Web Novel': 113, 'Horror': 789, 'Isolated Society': 23, 'Outside World': 22, 'Post-apocalyptic': 142, 'Body Sharing': 13, 'Exorcists': 26, 'Monsters': 237, 'School Life': 900, 'Urban Fantasy': 128, 'Sports': 929, 'Animeism': 30, 'School Club': 185, 'Tournaments': 139, 'Volleyball': 19, 'Seinen': 201

In [76]:
threshold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshold]
len(rare_genres), rare_genres[:5]

(599,
 ['Dark Fantasy',
  'War',
  'Animal Transformation',
  'Curse',
  'Dysfunctional Families'])

In [77]:
threshold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshold]
len(rare_genres), rare_genres[:5]

(599,
 ['Dark Fantasy',
  'War',
  'Animal Transformation',
  'Curse',
  'Dysfunctional Families'])

In [78]:
gegenres_list = df.Genre.to_list()
revised_genre_list = []
indices_to_drop = []

for idx, genres in enumerate(genres_list):
  genre_list = eval(genres)
  revised_genres = []

  for genre in genre_list:
    if genre not in rare_genres:
      revised_genres.append(genre)

  if len(revised_genres) == 0:
    indices_to_drop.append(idx)
  else:
    revised_genre_list.append(revised_genres)
df = df.drop(df.index[indices_to_drop]).reset_index(drop=True)
# df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape

(22582, 3)

In [79]:
df['revised_genres'] = revised_genre_list

In [80]:
revised_genres_list = df.revised_genres.to_list()
revised_genre_count = {}
for genres in revised_genres_list:
  genre_list = genres
  for genre in genre_list:
    if genre in revised_genre_count.keys():
      revised_genre_count[genre] += 1
    else:
      revised_genre_count[genre] = 1
print(f"Number of Genres: {len(revised_genre_count)}")
print(revised_genre_count)

Number of Genres: 56
{'Action': 7885, 'Drama': 4540, 'Fantasy': 6089, 'Shounen': 4957, 'Military': 1018, 'Based on a Manga': 2431, 'Romance': 4322, 'Shoujo': 1166, 'Contemporary Fantasy': 235, 'Adventure': 4613, 'Mystery': 1325, 'Demons': 604, 'Historical': 1288, 'Martial Arts': 582, 'Supernatural': 3068, 'Comedy': 9207, 'Chinese Animation': 547, 'Horror': 789, 'Monsters': 237, 'School Life': 900, 'Sports': 929, 'Seinen': 2015, 'Original Work': 1080, 'Superpowers': 361, 'Slice of Life': 3255, 'Psychological': 735, 'Mecha': 1609, 'Sci Fi': 1206, 'Aliens': 231, 'Parody': 579, 'Samurai': 348, 'Magic': 1888, 'Person in a Strange World': 257, 'Non-Human Protagonists': 885, 'Music': 591, 'Thriller': 292, 'Family Friendly': 734, 'Based on a Novel': 232, 'Based on a Light Novel': 535, 'CG Animation': 475, 'Ecchi': 1806, 'Based on a Video Game': 300, 'Short Episodes': 752, 'Animal Protagonists': 400, 'Anthropomorphic': 312, 'Harem': 1041, 'Police': 343, 'Shorts': 652, 'Promotional': 313, 'Super

In [23]:
categorical_genre_list = []
revised_genres_list = df.revised_genres.to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * len(encode_genre_types)
  for genre in revised_genres:
    genre_type_index = encode_genre_types[genre] 
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)



In [85]:
df['genre_cat_list'] = categorical_genre_list
df.head()

,Title,Description,Genre,revised_genres,genre_cat_list
0,Attack on Titan The Final Season: The Final Chapters,Final part of Attack on Titan The Final Season.,"['Action', 'Drama', 'Fantasy', 'Shounen', 'Dark Fantasy', 'Military', 'War', 'Based on a Manga']","[Action, Drama, Fantasy, Shounen, Military, Based on a Manga]","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Fruits Basket the Final Season,The final arc of Fruits Basket.,"['Drama', 'Fantasy', 'Romance', 'Shoujo', 'Animal Transformation', 'Contemporary Fantasy', 'Curse', 'Dysfunctional Families', 'Mental Illness', 'Orphans', 'Zodiac', 'Based on a Manga']","[Drama, Fantasy, Romance, Shoujo, Contemporary Fantasy, Based on a Manga]","[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Fullmetal Alchemist: Brotherhood,"The foundation of alchemy is based on the law of equivalent exchange; you cannot produce something from nothing. As such, alchemy is bound by one taboo - human transmutation. Four years ago two young brothers, Edward and Alphonse Elric, broke this taboo when they tried to resurrect their dead mother. During the process Al's body disintegrated and Ed lost his leg. In a desperate attempt to prevent his brother from disappearing completely, Ed sacrificed one of his arms so he could affix Al's soul to a suit of armor. When his missing limbs are replaced by auto mail parts, Ed bears the name of...","['Action', 'Adventure', 'Drama', 'Fantasy', 'Mystery', 'Shounen', 'Conspiracy', 'Death of a Loved One', 'Military', 'Siblings', 'Based on a Manga']","[Action, Adventure, Drama, Fantasy, Mystery, Shounen, Military, Based on a Manga]","[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,Demon Slayer: Kimetsu no Yaiba - Entertainment District Arc,"Tanjiro and his friends accompany the Hashira Tengen Uzui to an entertainment district where Tengen’s female ninja agents were gathering information on a demon before they suddenly disappeared. In order to investigate, Tanjiro and the others disguise themselves as women to sneak in!","['Action', 'Adventure', 'Shounen', 'Demons', 'Historical', 'Martial Arts', 'Orphans', 'Siblings', 'Supernatural', 'Swordplay', 'Based on a Manga']","[Action, Adventure, Shounen, Demons, Historical, Martial Arts, Supernatural, Based on a Manga]","[1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Bleach: Thousand-Year Blood War,"The peace is suddenly broken when warning sirens blare through the Soul Society. Residents there are disappearing without a trace and nobody knows who's behind it. Meanwhile, a dark shadow is also extending itself toward Ichigo and his friends in Karakura Town…","['Action', 'Comedy', 'Drama', 'Shounen', 'Afterlife', 'Ghosts', 'Shinigami', 'Supernatural', 'War', 'Based on a Manga']","[Action, Comedy, Drama, Shounen, Supernatural, Based on a Manga]","[1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [163]:
df=df.reset_index(drop=True)


In [164]:
df.shape

(21828, 5)

In [165]:
genres_list = df.Genre.to_list()
genre_count = {}
for genres in genres_list:
  genre_list = eval(genres)
  for genre in genre_list:
    if genre in genre_count.keys():
      genre_count[genre] += 1
    else:
      genre_count[genre] = 1
print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

threshold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshold]
len(rare_genres)

Number of Genres: 598
{'Comedy': 9124, 'Shounen': 4955, 'School': 2534, 'Romance': 4306, 'Ecchi': 1804, 'Seinen': 2015, 'Slice of Life': 3231, 'Shoujo': 1166, 'Shounen Ai': 261, 'Action': 7858, 'Martial Arts': 581, 'Fantasy': 6071, 'Supernatural': 3060, 'Horror': 787, 'Samurai': 348, 'Thriller': 292, 'Vampire': 346, 'Ninja': 57, 'Drama': 4519, 'GL': 37, 'Shoujo-ai': 31, 'Catholic School': 6, 'LGBT Themes': 38, 'School Life': 897, 'Based on a Light Novel': 534, 'BL': 78, 'Shounen-ai': 60, 'Adult Couples': 52, 'Blackmail': 10, 'Childcare': 54, 'Editors': 5, 'Mature Romance': 47, 'Salaryman': 14, 'Single Parent': 14, 'Tsundere': 40, 'Yaoi Hand Syndrome': 7, 'Based on a Manga': 2420, 'Sci-Fi': 2923, 'Space': 384, 'Sports': 916, 'Hand to Hand Combat': 111, 'Wrestling': 15, 'Adventure': 4586, 'Magic': 1881, 'Mecha': 1609, 'Crude': 64, 'Demons': 602, 'Episodic': 172, 'Family Friendly': 666, 'Korean Animation': 116, 'Monsters': 235, 'Monster School': 10, 'No Dialogue': 123, 'Short Episodes': 7

543

In [166]:
genres_list = df.Genre.to_list()
revised_genre_list = []
indices_to_drop = []

for idx, genres in enumerate(genres_list):
  genre_list = eval(genres)
  revised_genres = []

  for genre in genre_list:
    if genre not in rare_genres:
      revised_genres.append(genre)

  if len(revised_genres) == 0:
    indices_to_drop.append(idx)
  else:
    revised_genre_list.append(revised_genres)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_genres'] = revised_genre_list
df.shape

(21828, 5)

In [163]:
revised_genres_list = df.revised_genres.to_list()
revised_genre_count = {}
for genres in revised_genres_list:
  genre_list = genres
  for genre in genre_list:
    if genre in revised_genre_count.keys():
      revised_genre_count[genre] += 1
    else:
      revised_genre_count[genre] = 1
print(f"Number of Genres: {len(revised_genre_count)}")
print(revised_genre_count)

Number of Genres: 56
{'Action': 7885, 'Drama': 4540, 'Fantasy': 6089, 'Shounen': 4957, 'Military': 1018, 'Based on a Manga': 2431, 'Romance': 4322, 'Shoujo': 1166, 'Contemporary Fantasy': 235, 'Adventure': 4613, 'Mystery': 1325, 'Demons': 604, 'Historical': 1288, 'Martial Arts': 582, 'Supernatural': 3068, 'Comedy': 9207, 'Chinese Animation': 547, 'Horror': 789, 'Monsters': 237, 'School Life': 900, 'Sports': 929, 'Seinen': 2015, 'Original Work': 1080, 'Superpowers': 361, 'Slice of Life': 3255, 'Psychological': 735, 'Mecha': 1609, 'Sci Fi': 1206, 'Aliens': 231, 'Parody': 579, 'Samurai': 348, 'Magic': 1888, 'Person in a Strange World': 257, 'Non-Human Protagonists': 885, 'Music': 591, 'Thriller': 292, 'Family Friendly': 734, 'Based on a Novel': 232, 'Based on a Light Novel': 535, 'CG Animation': 475, 'Ecchi': 1806, 'Based on a Video Game': 300, 'Short Episodes': 752, 'Animal Protagonists': 400, 'Anthropomorphic': 312, 'Harem': 1041, 'Police': 343, 'Shorts': 652, 'Promotional': 313, 'Super

In [341]:
encode_genre_types = { key: idx for idx, (key, value) in enumerate(revised_genre_count.items())}
with open("genre_types_encoded.json", "w") as fp:
  json.dump(encode_genre_types, fp)

# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_genre_list = []
revised_genres_list = df.revised_genres.to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * len(encode_genre_types)
  for genre in revised_genres:
    genre_type_index = encode_genre_types[genre] 
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)

df['genre_cat_list'] = categorical_genre_list
df.shape

(21828, 5)

In [342]:
labels = list(encode_genre_types.keys())
len(labels), labels[:5]

(56, ['Drama', 'Historical', 'Slice of Life', 'Seinen', 'Romance'])

In [343]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(19646, 2182)

In [344]:
valid_df = df.loc[valid_ids]
valid_df.head()

,Title,Description,Genre,revised_genres,genre_cat_list
1254,E's Otherwise,"Kai, who has powers different from the rest, together with his sickly sister Hikaru was protected by an organization called ASHURUM. Scouted by Eiji, Kai was delegated to the ASHURUM special force AESES and undergo intensive training. Whenever he was free, Kai visited Hikaru at the hospital belongs to the organization, but Hikaru's condition never improved. So, one year later, with amazing growth from the intensive training, Kai decides to escape from the organization.","['Action', 'Adventure', 'Fantasy', 'Supernatural', 'Shounen']","[Action, Adventure, Fantasy, Supernatural, Shounen]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5742,Harmonie,"Every person has his or her own little world. Among the 34 bustling students of classroom 2-1, there are many such worlds. Akio Honjō's world — my world — is filled with passionate talk about last night's anime with Yoshida, Watanabe, and other friends. It's a fun, comfortable place…. But, I always wonder. Wonder about the world of Juri Makina, smiling and surrounded by gorgeous friends on the other side of the classroom. One day, I enter her world. But it's not as simple as it sounds.","['Adventure', 'Drama', 'Sci Fi', 'Monsters', 'Short Episodes']","[Adventure, Drama, Sci Fi, Monsters, Short Episodes]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
13320,Iya na Kao sare nagara Opantsu Misete Moraitai,Short anime about underwear!,"['Comedy', 'Slice of Life', 'Seinen']","[Comedy, Slice of Life, Seinen]","[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7567,Zom 100: Bucket List of the Dead,"Surviving a zombie apocalypse beats being a wage slave any day! After spending years slaving away for a soul-crushing company, Akira's life has lost its luster. He lives in a trash-filled apartment, his pay is abysmal, and he can't even muster up the nerve to confess his love to his beautiful co-worker. But when a zombie apocalypse ravages his town, it gives him the push he needs to live for himself. Now Akira's on a mission to complete all 100 items on his bucket list before he...well, kicks the bucket.","['Action', 'Comedy', 'Sci Fi', 'Hacking', 'Social Media', 'Virtual Reality', 'Original Work']","[Action, Comedy, Sci Fi, Original Work]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
11886,Zatch Bell Movie 1: 101st Devil,"While the battle to determine the next king of the demon realm has often been fraught, the rules were always clear: 100 demon children partnered with 100 humans would challenge each other, using 100 magical books. What none of them knew was that there was a 101st magical book, powerful enough to stop any of the others. Gash and Kiyomaro must begin a journey that will take them into the demon realm to confront the mysteries of Gash's past. But will they escape in time to help their friends in a battle with a seemingly unstoppable enemy.","['Comedy', 'Ecchi', 'Seinen', 'Age Gap', 'Recap', 'Student-Teacher Relationship', 'Based on a Manga']","[Comedy, Ecchi, Seinen, Based on a Manga]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


# Fastai & Blurr Inference

In [171]:
model_path="models/cartoon-classifier-stage-1.pkl"
learner_inf=load_learner(model_path)

In [172]:
learner_inf.blurr_predict("random placeholder")

[{'labels': [],
  'scores': [],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['Comedy', 'Shounen', 'School', 'Romance', 'Ecchi', 'Seinen', 'Slice of Life', 'Shoujo', 'Shounen Ai', 'Action', 'Martial Arts', 'Fantasy', 'Supernatural', 'Horror', 'Samurai', 'Thriller', 'Vampire', 'Drama', 'School Life', 'Based on a Light Novel', 'Based on a Manga', 'Sci-Fi', 'Space', 'Sports', 'Adventure', 'Magic', 'Mecha', 'Demons', 'Family Friendly', 'Monsters', 'Short Episodes', 'CG Animation', 'Mystery', 'Animal Protagonists', 'Non-Human Protagonists', 'Sci Fi', 'Original Work', 'Psychological', 'Game', 'Super Power', 'Historical', 'Shorts', 'Parody', 'Harem', 'Superpowers', 'Milit

In [104]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['genre_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [106]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['Description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_genres = [0] * len(encode_genre_types)
  for label in labels:metric_measures(valid_df, preds)
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

preds[0][:20]

  0%|          | 0/2258 [00:00<?, ?it/s]

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [107]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.17534811758638474
F1 Score (Macro) = 0.017750645321062818


# Convert to ONNX

In [108]:
model_path = "models/cartoon-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [114]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier, 
    torch.LongTensor([[0] * 512]),
    'models/cartoon-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [120]:
metric_measures(valid_df, preds) 

F1 Score (Micro) = 0.17534811758638474
F1 Score (Macro) = 0.017750645321062818


In [119]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2258 [00:00<?, ?it/s]

In [118]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/cartoon-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [116]:
## Normal ONNX

# ONNX Inference

In [115]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'models/cartoon-classifier.onnx'
quantized_onnx_model_path = 'models/cartoon-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

In [112]:
pip install onnxruntime


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 28.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [122]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/cartoon-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [125]:
metric_measures(valid_df, preds) 

F1 Score (Micro) = 0.17534811758638474
F1 Score (Macro) = 0.017750645321062818
